# Part 1: Option Valuation

## Ex. 1: Binomial tree program to approximate the price of the option. 
How does your estimate compare to the analytical value? <br>
Experiment for different values of the volatility

In [1]:
import math

stock_price = 100.0
strike_price = 99.0
interest_rate = 0.06
volatility = 0.20
steps = 50 
T = 1 # tijd uitoefenen 1 jaar

d = math.exp(-volatility * math.sqrt(1/steps))
u  = math.exp(volatility * math.sqrt(1/steps))
a = math.exp(interest_rate * 1/steps )
p = (a - d) / (u - d)

def binomial_tree(steps, start):
    ''' Binomial tree to approximate the price of the option ''' 
    # Result includes stock price, option price, delta
    results = [[[start, 0, 0]]]
    
    for i in range(steps): 
        result = []
        for j in range(len(results[-1])):
            if j == 0:
                result.append([results[-1][j][0] * d, 0, 0])
            result.append([results[-1][j][0] * u, 0, 0])
        results.append(result)  
   
    return results


def call_option(S, K):
    ''' Calculates the value of a call option at its expiration date '''
    return max(0, S - K)

def risk_neutral_valuation(r, dt, fi, fj): 
    ''' Risk-neutral valuation of the option '''
    return math.exp(-r * dt) * (p * fi + (1 - p)*fj)

def price_call(treelist, strike_price):
    lentree = len(treelist)
    for el in treelist[-1]:
        el[1] = call_option(el[0], strike_price)
        
    for j in range(lentree - 1):
        for k in range(len(treelist[-(j+2)])):
            treelist[-(j+2)][k][1] = risk_neutral_valuation(interest_rate, 1/steps, treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1])
#             if j == 0 and k ==0:
#                 print(treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1], treelist[-(j+2)][k][1])
    return treelist

def calc_delta(treelist):
    lentree = len(treelist)
        
    for j in range(lentree - 1):
        for k in range(len(treelist[-(j+2)])):
            treelist[-(j+2)][k][2] = (treelist[-(j+1)][k][1] - treelist[-(j+1)][k+1][1]) / (treelist[-(j+1)][k][0] - treelist[-(j+1)][k+1][0])
#             if j == 0 and k ==0:
#                 print(treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1], treelist[-(j+2)][k][1])
    return treelist


In [2]:
''' The estimate compared to the analytical value '''
#The estimate 
treelist = binomial_tree(steps, stock_price)
filled_tree = price_call(treelist, strike_price)
print(filled_tree[0][0][1])

#The analytical value


6.9413989712433875


In [3]:
''' Different values of volatility '''
import matplotlib.pyplot as plt

plt.figure()
plt.title('Approximation of the option price')
plt.show()

<Figure size 640x480 with 1 Axes>

## Ex. 2: Convergence of the method
What is the computational complexity of this algorithm? <br>
Done

## Ex. 3: Hedge parameter
Compare with the analytical values. 
Experiment for different values of the volatility.

In [4]:
better_tree = calc_delta(filled_tree)
print(better_tree[0])

[[100.0, 6.9413989712433875, 0.48108165285175314]]


## Ex. 4: American option 
What is the value of the American put and call for the corresponding parameters? <br>
Experiment for different values of the volatility.

In [10]:
from scipy.stats import norm


d1 = (math.log(stock_price / strike_price) + (interest_rate + volatility ** 2 / 2) * T)
d2 = d1 - volatility* math.sqrt(T)


call = stock_price*norm.cdf(d1) - strike_price*math.exp(-interest_rate * T)*norm.cdf(d2)
put = strike_price*math.exp(-interest_rate * T ) * norm.cdf(-d2) - stock_price*norm.cdf(-d1)

print(call, put)

11.051676204724757 4.286365029565374


In [11]:
def price_US_call(treelist, strike_price):
    lentree = len(treelist)
    for el in treelist[-1]:
        el[1] = call_option(el[0], strike_price)
        
    for j in range(lentree - 1):
        for k in range(len(treelist[-(j+2)])):
            keeping_value = risk_neutral_valuation(interest_rate, 1/steps, treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1])
            use_now_value = call_option(treelist[-(j+2)][k][0], strike_price)
            
            treelist[-(j+2)][k][1] = max(keeping_value, use_now_value)
#             if j == 0 and k ==0:
#                 print(treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1], treelist[-(j+2)][k][1])
    return treelist

treelist = binomial_tree(steps, stock_price)
print(price_US_call(treelist, strike_price))

[[[100.0, 7.289709909037599, 0]], [[97.21119840328973, 5.882631299027478, 0], [102.86880693018583, 8.796779343743, 0]], [[94.50017095003759, 4.671525194025435, 0], [100.0, 7.178833609085029, 0], [105.81991439239847, 10.530725386009474, 0]], [[91.864748673689, 3.6448516238599016, 0], [97.21119840328971, 5.7695569826329995, 0], [102.86880693018583, 8.687955095656378, 0], [108.85568343000432, 12.50687931511549, 0]], [[89.30282309586327, 2.789182376119669, 0], [94.50017095003759, 4.559379223875268, 0], [99.99999999999999, 7.064497410741267, 0], [105.81991439239847, 10.427465261715671, 0], [111.97854282014544, 14.738347505354144, 0]], [[86.81234453945848, 2.0893822272072784, 0], [91.864748673689, 3.5366453940660616, 0], [97.21119840328971, 5.652964589726276, 0], [102.86880693018581, 8.575723965932426, 0], [108.85568343000432, 12.412857642245152, 0], [115.19099101689088, 17.235742487587768, 0]], [[84.39132048880043, 1.5289288520626232, 0], [89.30282309586326, 2.687650777212299, 0], [94.50017

# Part 2: Hedging Simulations

In [ ]:
# T = 1

stock_price = 100.0
strike_price = 99.0
interest_rate = 1.06
volatility = 0.20

def dynamics_stock_price(r, S, dt, sigma, dZ):
    return r*S*dt + sigma * S * dZ

